In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import zipfile
from tqdm.notebook import tqdm

In [2]:
zip_data = 'D:/Project/Brain Tumors Classification with PyTorch/data.zip'
best_model_data = 'D:/Project/Brain Tumors Classification with PyTorch/best_model.pth'
extract_path = 'D:/Project/Brain Tumors Classification with PyTorch/'

os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_data, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path + 'data/')

['Brain Tumor Data Set', 'metadata.csv', 'metadata_rgb_only.csv']

In [3]:
print(f"Cek GPU: ", "Terhubung" if torch.cuda.is_available() else "Tidak ada GPU")

Cek GPU:  Terhubung


In [5]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64*64*32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.maxpool(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.relu(x)
        print(x.shape)
        x = self.maxpool(x)
        print(x.shape)
        x = torch.flatten(x, 1)
        print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
sample_input = torch.randn(1, 3, 256, 256)
model = SimpleCNN()
model(sample_input)

torch.Size([1, 16, 256, 256])
torch.Size([1, 16, 256, 256])
torch.Size([1, 16, 128, 128])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 128, 128])
torch.Size([1, 32, 64, 64])
torch.Size([1, 131072])


tensor([[-0.4808, -0.9631]], grad_fn=<LogSoftmaxBackward0>)

In [7]:
data_dir = "./data/"
metadata = pd.read_csv(data_dir + "metadata.csv").drop(columns="Unnamed: 0")

In [8]:
metadata.head()

,image,class,format,mode,shape
0,Cancer (1).jpg,tumor,JPEG,RGB,"(512, 512, 3)"
1,Cancer (1).png,tumor,PNG,L,"(300, 240)"
2,Cancer (1).tif,tumor,TIFF,RGB,"(256, 256, 3)"
3,Cancer (10).jpg,tumor,JPEG,RGB,"(512, 512, 3)"
4,Cancer (10).tif,tumor,TIFF,RGB,"(256, 256, 3)"


In [9]:
transform = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
])

dataset = torchvision.datasets.ImageFolder(root=data_dir + "Brain Tumor Data Set", transform=transform)

In [10]:
train_size = int(0.7 * len(dataset))
eval_size = len(dataset) - train_size
val_size = int(0.5 * eval_size)
test_size = eval_size - val_size

train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])
val_dataset, test_dataset = random_split(eval_dataset, [val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [11]:
# Model, loss function, optimizer and scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Gunakan GPU jika tersedia
model = SimpleCNN(num_classes=2).to(device)

criterion = nn.NLLLoss(reduction="sum") # NLLLoss menerima input log softmax, sedikit berbeda dengan CrossEntropyLoss. CrossEntropyLoss akan menghitung softmax dari input dan kemudian menghitung log loss
optimizer = optim.Adam(model.parameters(), lr=3e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.5, patience=20,verbose=1) # Scheduler digunakan untuk mengurangi learning rate jika validation loss tidak berkurang dalam beberapa epoch, ini akan membantu model untuk mencapai konvergensi

# Training and validation loop
num_epochs = 60
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train() # Set model ke mode training
    # Inisiasi running loss, correct, dan total menjadi nol dalam satu epoch
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad() # Memastikan gradient pada model di set menjadi nol sebelum melakukan backpropagation
        outputs = model(inputs) # Forward propagation, menghitung output dari input
        loss = criterion(outputs, labels) # Menghitung loss dengan membandingkan output dengan label
        loss.backward() # Backward propagation, menghitung gradient dari loss terhadap model
        optimizer.step() # Update parameter model berdasarkan gradient yang dihitung sebelumnya

        running_loss += loss.item() * inputs.size(0) # Menghitung total loss dalam satu epoch
        _, predicted = torch.max(outputs, 1) # Menghitung prediksi dari output model
        total += labels.size(0) # Menghitung total data
        correct += (predicted == labels).sum().item() # Menghitung total prediksi yang benar

    epoch_loss = running_loss / len(train_loader.dataset) # Menghitung rata-rata loss dalam satu epoch
    epoch_acc = correct / total # Menghitung akurasi dalam satu epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')

    # Validation phase
    model.eval() # Set model ke mode evaluasi, model tidak akan melakukan backpropagation dalam mode ini
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    # Simpan model dengan validation accuracy terbaik
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

    # Step the scheduler
    scheduler.step(val_loss)

print('Finished Training')

d:\App\Anaconda\envs\py3.10\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


  0%|          | 0/101 [00:00<?, ?it/s]

torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Siz

  0%|          | 0/101 [00:00<?, ?it/s]

torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Siz

...

torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Siz

  0%|          | 0/101 [00:00<?, ?it/s]

torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Siz

  0%|          | 0/101 [00:00<?, ?it/s]

torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Siz

In [14]:
val_loss = 0.0
correct = 0
total = 0

all_labels = []
all_predictions = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

test_acc = correct / total
average_val_loss = val_loss / total

print(f'Test Accuracy : {test_acc:.4f}')
print(f'Average Validation Loss : {average_val_loss:.4f}')

conf_matrix = confusion_matrix(all_labels, all_predictions)
class_report = classification_report(all_labels,all_predictions)

print('Confusion Matrix : ')
print(conf_matrix)
print('\nClassification Report : ')
print(class_report)

torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 64, 64])
torch.Size([32, 131072])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 256, 256])
torch.Size([32, 16, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Size([32, 32, 128, 128])
torch.Siz